In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
########################
###   experimental   ###
########################

In [3]:
path = "/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s1_raster/march/S1_sampled_original.csv"
data_org = pd.read_csv(path)

In [4]:
data_org = data_org.drop('.geo', axis=1)

In [5]:
data_org['system:index'] = data_org['system:index'].astype('str')
# extract date
data_org['date'] = data_org['system:index'].str[17:25]
data_org['poi'] = data_org['system:index'].str[85:-2]

In [6]:
scenes = data_org['date'].unique()

In [7]:
data_org

,system:index,Crop_ID_in,VH,VV,id,date,poi
0,S1A_IW_GRDH_1SDV_20180401T050947_20180401T0510...,2.0,-21.051089,-4.754000,100,20180401,1fc
1,S1A_IW_GRDH_1SDV_20180401T050947_20180401T0510...,2.0,-18.604523,-7.891009,101,20180401,1fd
2,S1A_IW_GRDH_1SDV_20180401T050947_20180401T0510...,2.0,-23.217787,-9.946385,102,20180401,1fe
3,S1A_IW_GRDH_1SDV_20180401T050947_20180401T0510...,2.0,-14.366249,-7.015451,103,20180401,1ff
4,S1A_IW_GRDH_1SDV_20180401T050947_20180401T0510...,2.0,-17.425313,-8.078750,104,20180401,200
...,...,...,...,...,...,...,...
115213,S1B_IW_GRDH_1SDV_20180918T165013_20180918T1650...,1.0,-19.739376,-12.201721,89,20180918,1f1
115214,S1B_IW_GRDH_1SDV_20180918T165013_20180918T1650...,1.0,-20.389221,-13.185935,93,20180918,1f5
115215,S1B_IW_GRDH_1SDV_20180918T165013_20180918T1650...,1.0,-17.652814,-17.133226,96,20180918,1f8
115216,S1B_IW_GRDH_1SDV_20180918T165013_20180918T1650...,1.0,-25.073906,-12.158048,98,20180918,1fa


In [9]:
s1_list = []
set_crop_id = 0
for scene in scenes:
    # filter dataframe
    temp = data_org.loc[data_org['date']==scene,['id', 'Crop_ID_in', 'VH', 'VV']]
    # add just tables with all pois
    if temp.shape[0] == 1200:
        temp = temp.set_index('id')
        if set_crop_id == 0:
            temp.rename(columns={'Crop_ID_in': 'crop_id'}, inplace=True)
            s1_list.append(temp['crop_id'])
            set_crop_id = 1
        temp = temp[['VH', 'VV']]
        temp.columns = ['VH_'+scene, 'VV_'+scene]
        #temp = temp.reset_index(drop=True)
        s1_list.append(temp)

In [10]:
result = pd.concat(s1_list, axis=1)

In [11]:
result.columns

Index(['crop_id', 'VH_20180404', 'VV_20180404', 'VH_20180408', 'VV_20180408',
       'VH_20180416', 'VV_20180416', 'VH_20180420', 'VV_20180420',
       'VH_20180428',
       ...
       'VH_20180824', 'VV_20180824', 'VH_20180901', 'VV_20180901',
       'VH_20180905', 'VV_20180905', 'VH_20180913', 'VV_20180913',
       'VH_20180917', 'VV_20180917'],
      dtype='object', length=115)

In [12]:
result

,crop_id,VH_20180404,VV_20180404,VH_20180408,VV_20180408,VH_20180416,VV_20180416,VH_20180420,VV_20180420,VH_20180428,...,VH_20180824,VV_20180824,VH_20180901,VV_20180901,VH_20180905,VV_20180905,VH_20180913,VV_20180913,VH_20180917,VV_20180917
id,,,,,,,,,,,,,,,,,,,,,
100,2.0,-27.494673,-11.728935,-26.157115,-7.941369,-36.184080,-16.116791,-29.513807,-13.442985,-46.171668,...,-11.724801,-8.954501,-12.076220,-5.997058,-12.227370,-11.059301,-13.430295,-6.561589,-14.848242,-7.553686
101,2.0,-21.734028,-11.113761,-20.046772,-14.004034,-22.975343,-15.385728,-24.446698,-16.655647,-25.314481,...,-22.096094,-16.113720,-16.285748,-9.972386,-13.234767,-10.703640,-17.659846,-12.453556,-26.056990,-15.490167
102,2.0,-24.093901,-16.089591,-24.576685,-20.076986,-31.567238,-16.418191,-24.024340,-15.141388,-24.571568,...,-16.124723,-10.424468,-15.833768,-7.069556,-15.174808,-8.086657,-16.570060,-9.753053,-15.719652,-6.077555
103,2.0,-36.676221,-18.670664,-25.934613,-15.942835,-47.155393,-18.166004,-25.290986,-20.948154,-21.134109,...,-15.879471,-10.932715,-12.135976,-10.111052,-13.671813,-5.494979,-15.225784,-7.806783,-13.673005,-7.752452
104,2.0,-35.661983,-17.770296,-34.846377,-18.229650,-26.195068,-15.378349,-23.733361,-17.265222,-26.952289,...,-16.923059,-10.823628,-16.064799,-5.314717,-13.808311,-7.630617,-17.314107,-12.517806,-15.498072,-12.025849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,-24.775502,-12.593654,-26.455746,-14.772138,-23.196135,-10.629935,-24.705778,-15.229416,-22.031368,...,-28.955559,-15.446224,-17.937402,-8.133442,-19.399663,-9.802850,-34.663145,-13.007374,-32.655200,-10.394704
96,1.0,-36.307976,-17.293078,-23.119823,-14.799103,-27.757228,-15.858848,-29.653040,-13.716123,-29.801011,...,-22.444245,-12.963899,-12.504291,-7.625903,-16.328767,-9.162626,-25.312933,-14.088199,-20.431067,-15.957579
97,1.0,-24.827906,-13.117438,-24.963632,-15.799011,-24.136955,-14.889866,-24.613887,-18.657440,-27.864236,...,-21.481152,-12.098764,-13.464127,-8.363618,-16.183364,-7.536353,-22.472052,-13.107223,-24.657991,-14.042103


In [13]:
result.to_csv("/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/crops/s1_raster/march/S1_sampled_clean.csv")